In [1]:
from Bio import SeqIO
from Bio import motifs
def load_motif(file_meme_xml, motif_index):
    meme_record = motifs.parse(open(file_meme_xml), 'meme')
    motif = meme_record[motif_index]
    motif.pseudocounts = 1
    return motif
motivo= load_motif('/home/davide/Desktop/genomiChro/MEME/motivo8_oops/meme.xml', 0)
print(motivo.counts)

        0      1      2      3      4      5      6      7      8      9     10     11     12     13     14     15
A:   7.00  99.00   0.00   0.00  39.00  25.00  43.00  41.00  19.00  15.00   1.00  12.00  77.00   0.00   0.00  64.00
C:  24.00   0.00   0.00   0.00  15.00  55.00  12.00   7.00  26.00   2.00   4.00  22.00  12.00  99.00   0.00  13.00
G:   6.00   0.00  99.00   0.00  17.00   9.00  32.00   0.00  10.00   6.00  64.00  17.00   0.00   0.00   0.00  21.00
T:  62.00   0.00   0.00  99.00  28.00  10.00  12.00  51.00  44.00  76.00  30.00  48.00  10.00   0.00  99.00   1.00


In [12]:
import numpy as np
from math import log
def precompute_values(motif):
    q = motif.background
    pwm = np.array([motif.pwm[base] for base in "ACGT"])

    n = motif.num_occurrences
    a = (n + 1) / (n + 4) * log(n + 1) - log(n + 4) - 1 / (n + 4) * sum(log(q[b]) for b in "ACGT") - n / (n + 4) * log(min(q.values()))

    relative_entropy = np.zeros(motif.length)
    for i in range(motif.length):
        relative_entropy[i] = np.sum(pwm[:, i] * np.log(pwm[:, i] / np.array([q[b] for b in "ACGT"]))) / a

    return a, relative_entropy

for i in range(10000):
    a, relative_entropy = precompute_values(motivo)


In [13]:
def precompute_values2(motif):
    q = motif.background
    pwm = motif.pwm

    n = motif.num_occurrences
    a = (n + 1) / (n + 4) * log(n + 1) - log(n + 4) - 1 / (n + 4) * sum(log(q[b]) for b in "ACGT") - n / (n + 4) * log(min(q.values()))
    def Info(i):
        """ Calcola l'entropia relativa per la posizione i del motivo"""
        somma = sum(pwm[b, i] * log(pwm[b, i] / q[b]) for b in "ACGT")
        return somma / a
    relative_entropy = [Info(i) for i in range(motif.length)]
    return a, relative_entropy
for i in range(10000):
    a, relative_entropy = precompute_values2(motivo)
print(a)
print(relative_entropy)

1.2226043884126738
[0.29506897441071567, 0.9999999999999998, 0.9999999999999998, 0.9999999999999998, 0.05563566338923674, 0.1980821349513933, 0.11108714605020387, 0.3533247096368185, 0.0952140136172144, 0.4856962387796685, 0.4179641373130755, 0.10798075931297677, 0.515815240045664, 0.9999999999999998, 0.9999999999999998, 0.34211891465231736]


In [15]:
def sm_score(motif, seq):
    q = motif.background
    pwm = np.array([motif.pwm[base] for base in "ACGT"])

    def normalization_factor():
        n = motif.num_occurrences
        return (n + 1) / (n + 4) * log(n + 1) - log(n + 4) - 1 / (n + 4) * sum(log(q[b]) for b in "ACGT") - n / (
                    n + 4) * log(min(q.values()))

    a = normalization_factor()

    def relative_entropy(i):
        return np.sum(pwm[:, i] * np.log(pwm[:, i] / np.array([q[b] for b in "ACGT"]))) / a

    max_score = -float("inf")
    max_index = 0
    motif_len = motif.length
    seq_len = len(seq)

    for i in range(seq_len - motif_len + 1):
        sub_seq = seq[i:i + motif_len]
        sub_seq_indices = np.array([np.where(np.array(list("ACGT")) == base)[0][0] for base in sub_seq])
        score = np.sum(
            [relative_entropy(j) * log(pwm[sub_seq_indices[j], j] / q[sub_seq[j]]) for j in range(motif_len)])
        if score > max_score:
            max_score = score
            max_index = i

    return max_score, seq[max_index:max_index + motif_len], max_index - seq_len
seq = "A"*motivo.length
score, sub_seq, position = sm_score(motivo, seq)
for i in range(1000):
    score, sub_seq, position = sm_score(motivo, seq)


In [19]:
import numpy as np
from math import log
import Bio
def sm(motivo: Bio.motifs, seq: str):
    """
    Calcola lo score SM di una sequenza rispetto ad un motivo
    :param motivo:  il motivo in formato Bio.motifs
    :param seq:     la sequenza su cui calcolare lo score
    :return:    lo score migliore nella sequenza intergenica rispetto al motivo, la sottosequenza a cui corrisponde e la usa poszione rispetto a inizio trascrizione
    """
    q = motivo.background  # frequenze delle basi in tutte le sequenze intergeniche
    pwm = motivo.pwm  # matrice di probabilità delle basi per ogni posizione del motivo

    def a():
        """
        Calcola il fattore di normalizzazione a
        :return:    il fattore di normalizzazione a
        """
        n = motivo.num_occurrences  # numero di sequenze con cui è stato costruito il motivo
        a = (n + 1) / (n + 4) * log(n + 1) - log(n + 4) - 1 / (n + 4) * sum(log(q[b]) for b in "ACGT") - n / (
                    n + 4) * log(min(q.values()))
        return a

    a = a()  # fattore di normalizzazione a

    def Info(i):
        """ Calcola l'entropia relativa per la posizione i del motivo"""
        somma = sum(pwm[b, i] * log(pwm[b, i] / q[b]) for b in "ACGT")
        return somma / a

    max = -float("inf")
    max_i = 0
    for i in range(len(seq) - motivo.length + 1):
        h = seq[i:i + motivo.length]  # sottosequenza di lunghezza del motivo(l-mero della sequenza intergenica)
        score = sum(Info(i) * log(pwm[h[i], i] / q[h[i]]) for i in range(len(h)))  # score della sottosequenza
        if score > max:
            max = score
            max_i = i
    return max, seq[max_i:max_i + motivo.length], max_i - len(seq)
seq = "A"*motivo.length
score, sub_seq, position = sm(motivo, seq)
for i in range(1000):
    seq="".join(np.random.choice(list("ACGT"), motivo.length))
    score, sub_seq, position = sm(motivo, seq)

In [20]:
def hamming_distance(s1, s2):
    return np.sum(np.array(list(s1)) != np.array(list(s2)))


def ortholog_sequences(pid, intergen_dir, genome_ref):
    seqs = []
    for file in intergen_dir.iterdir():
        if genome_ref not in file.name:
            for record in SeqIO.parse(file, "fasta"):
                if pid in record.description:
                    seqs.append(record.seq)
    return seqs


def refined_score(motif, pid, intergenic_seq, intergen_dir, genome_ref):
    orthologs = ortholog_sequences(pid, intergen_dir, genome_ref)
    sm_result = sm_score(motif, intergenic_seq)
    s = str(sm_result[1])
    motif_len = motif.length

    ortholog_scores = [
        (motif_len - hamming_distance(s, sm_score(motif, seq)[1])) / motif_len * sm_score(motif, seq)[0]
        for seq in orthologs if len(seq) >= motif_len and "N" not in seq
    ]
    avg_ortholog_score = np.mean(ortholog_scores) if ortholog_scores else 0
    return sm_result[0], sm_result[0] + avg_ortholog_score, s, sm_result[2]

In [24]:
from pathlib import Path
intergen_dir = Path("/home/davide/Desktop/genomiChro/intergeniche_RefSeq/ortologhi")
genome_ref = "CCMEE_29"
for intergen_file in intergen_dir.iterdir():
    if genome_ref in intergen_file.name:
        intergen_path = str(intergen_file)
        break
print(intergen_path)
for record in SeqIO.parse(intergen_path, "fasta"):
    if len(record.seq) < len(motivo) or "N" in record.seq:
        continue
    start = record.description.find("WP_")
    end = record.description.find("'", start)
    pid = record.description[start:end]
    result = refined_score(motivo, pid, record.seq, intergen_dir, genome_ref)
    print(result)
    break

/home/davide/Desktop/genomiChro/intergeniche_RefSeq/ortologhi/Chroococcidiopsis_sp._CCMEE_29_GCF_023558375_intergen.fasta
(2.6811825081050005, 4.266467839766377, 'CAGTGTAATTGCTCGG', -184)


In [32]:
i=0
for record in SeqIO.parse(intergen_path, "fasta"):
    if len(record.seq) < len(motivo) or "N" in record.seq:
        continue
    start = record.description.find("WP_")
    end = record.description.find("'", start)
    pid = record.description[start:end]
    result = refined_score(motivo, pid, record.seq, intergen_dir, genome_ref)
    i+=1
    if i==100:
        break

KeyboardInterrupt: 

In [30]:
40*5000/100

2000.0

In [31]:
2000/60

33.333333333333336